## **ENVIRONMENTAL SUSTAINABILITY ANALYSIS**

In [1]:
%run ./_setup.py
import os
import sys

Python: c:\Users\Admin\Desktop\Irrigation Water Requirement Prediction\.venv\Scripts\python.exe
Environment ready
Shape: (10000, 20)
<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Soil_Type                10000 non-null  str    
 1   Soil_pH                  10000 non-null  float64
 2   Soil_Moisture            10000 non-null  float64
 3   Organic_Carbon           10000 non-null  float64
 4   Electrical_Conductivity  10000 non-null  float64
 5   Temperature_C            10000 non-null  float64
 6   Humidity                 10000 non-null  float64
 7   Rainfall_mm              10000 non-null  float64
 8   Sunlight_Hours           10000 non-null  float64
 9   Wind_Speed_kmh           10000 non-null  float64
 10  Crop_Type                10000 non-null  str    
 11  Crop_Growth_Stage        10000 non-null  str    
 12  Season   

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
df = pd.read_csv("../data/irrigation_prediction.csv")

In [7]:
# Crop prices (example market prices)
crop_prices = {
    'Wheat': 250, 'Maize': 220, 'Cotton': 500, 
    'Rice': 300, 'Sugarcane': 280, 'Potato': 200
}
df['Crop_Price'] = df['Crop_Type'].map(crop_prices)

### *86. Carbon Footprint Analysis*

In [9]:
print("\n86. Carbon Footprint...")

# Assign carbon coefficients (kg CO2 per mm irrigation)
carbon_coeff = {'Drip': 0.3, 'Sprinkler': 0.5, 'Canal': 0.2, 'Rainfed': 0}
df['Carbon_Footprint'] = df['Previous_Irrigation_mm'] * df['Irrigation_Type'].map(carbon_coeff)

df["Crop_Price"] = pd.to_numeric(df["Crop_Price"], errors="coerce")
df["Previous_Irrigation_mm"] = pd.to_numeric(df["Previous_Irrigation_mm"], errors="coerce")

df["Revenue_per_ha"] = df["Crop_Price"] * 10  # Simplified yield
df["Profit_per_ha"] = df["Revenue_per_ha"] - df["Previous_Irrigation_mm"] * 0.1  # Cost of water
df["Water_Productivity"] = df["Revenue_per_ha"] / (df["Previous_Irrigation_mm"] + 1)

carbon_stats = df.groupby(['Crop_Type', 'Irrigation_Type'], observed=False).agg({
    'Carbon_Footprint': 'mean',
    'Water_Productivity': 'mean'
}).reset_index()

fig86 = px.scatter(carbon_stats, x='Carbon_Footprint', y='Water_Productivity',
                   color='Irrigation_Type', size='Carbon_Footprint',
                   facet_col='Crop_Type', facet_col_wrap=3,
                   title='<b>86. Carbon vs Water Efficiency</b>',
                   labels={'Carbon_Footprint': 'Carbon Footprint (kg CO2/ha)',
                          'Water_Productivity': 'Water Productivity'})

fig86.update_layout(height=700, plot_bgcolor='black')
fig86.show()


86. Carbon Footprint...


### *87. Biodiversity Impact*

In [13]:
print("\n87. Biodiversity Analysis...")

# Use crop diversity as proxy for biodiversity
biodiversity = df.groupby('Region', observed=False).agg(
    Crop_Type=('Crop_Type', 'nunique'),
    Mulching_Rate=('Mulching_Used', lambda x: (x == 'Yes').mean() * 100),
    Modern_Rate=('Irrigation_Type', lambda x: x.isin(['Drip', 'Sprinkler']).mean() * 100)
).reset_index()

fig87 = px.scatter(
    biodiversity,
    x='Crop_Type',
    y='Mulching_Rate',
    color='Region',
    size='Modern_Rate',
    title='<b>87. Biodiversity & Conservation Indicators</b><br>Size = Modern Methods',
    labels={
        'Crop_Type': 'Crop Diversity (# of crops)',
        'Mulching_Rate': 'Mulching Adoption (%)'
    }
)

fig87.update_layout(height=500, plot_bgcolor='black')
fig87.show()



87. Biodiversity Analysis...


### *88. Soil Erosion Risk*

In [ ]:
print("\n88. Soil Erosion Risk...")

# Erosion risk factors
df['Erosion_Risk'] = (
    (df['Soil_Type'] == 'sandy').astype(int) * 0.4 +
    (df['Rainfall_mm'] > 50).astype(int) * 0.3 +
    (df['Wind_Speed_kmh'] > 20).astype(int) * 0.3
) * 100

erosion_risk = df.groupby(['Region', 'Soil_Type'], observed=False).agg(
    Erosion_Risk=("Erosion_Risk", "mean"),
    Mulching_Rate=("Mulching_Used", lambda x: (x == "Yes").mean() * 100)
).reset_index()

fig88 = px.bar(erosion_risk, x='Region', y='Erosion_Risk', color='Soil_Type',
               barmode='group', title='<b>88. Soil Erosion Risk by Region</b>',
               labels={'Erosion_Risk': 'Erosion Risk Score'})

fig88.update_layout(height=500, plot_bgcolor='white')
fig88.show()


88. Soil Erosion Risk...


### *Pollution risk factors*

In [ ]:
print("\n89. Water Pollution Risk...")

# Pollution risk factors
df['Pollution_Risk'] = (
    (df['Electrical_Conductivity'] > 2).astype(int) * 0.4 +  # Salinity
    (df['Previous_Irrigation_mm'] > 100).astype(int) * 0.3 +  # Over-irrigation
    (df['Soil_Type'] == 'Sandy').astype(int) * 0.3  # Sandy soil = high leaching
) * 100

pollution_map = df.groupby(['Region', 'Crop_Type'], observed=False).agg(
    Pollution_Risk = ('Pollution_Risk', 'mean')).reset_index()

fig89 = px.density_heatmap(pollution_map, x='Region', y='Crop_Type', z='Pollution_Risk',
                           color_continuous_scale='Reds',
                           title='<b>89. Water Pollution Risk Matrix</b>',
                           labels={'Pollution_Risk': 'Pollution Risk Score'})

fig89.update_layout(height=500, plot_bgcolor='white')
fig89.show()


89. Water Pollution Risk...


### *90. Energy Consumption*

In [26]:
print("\n90. Energy-Water Nexus...")

# Energy consumption by irrigation type
energy_coeff = {'Drip': 0.8, 'Sprinkler': 1.2, 'Canal': 0.3, 'Rainfed': 0}
df['Energy_Use'] = df['Previous_Irrigation_mm'] * df['Irrigation_Type'].map(energy_coeff)

energy_stats = df.groupby(['Region', 'Irrigation_Type'], observed=False).agg({
    'Energy_Use': 'mean',
    'Water_Productivity': 'mean'
}).reset_index()

fig90 = px.scatter(energy_stats, x='Energy_Use', y='Water_Productivity',
                   color='Irrigation_Type', facet_col='Region', facet_col_wrap=2,
                   title='<b>90. Energy Use vs Water Productivity</b>',
                   labels={'Energy_Use': 'Energy Use Index', 
                          'Water_Productivity': 'Water Productivity'})

fig90.update_layout(height=600, plot_bgcolor='black')
fig90.show()


90. Energy-Water Nexus...


### *91. Waste Management*

In [27]:
print("\n91. Agricultural Waste...")

# Waste generation coefficients (tonnes per hectare)
waste_coeff = {'Wheat': 1.2, 'Maize': 1.5, 'Cotton': 2.0, 
               'Rice': 1.8, 'Sugarcane': 3.0, 'Potato': 1.0}

waste_data = df.groupby(['Crop_Type', 'Region'], observed=False).agg({
    'Field_Area_hectare': 'sum'
}).reset_index()

waste_data['Waste_Generated'] = waste_data['Crop_Type'].map(waste_coeff) * waste_data['Field_Area_hectare']

fig91 = px.treemap(waste_data, path=['Region', 'Crop_Type'], 
                   values='Waste_Generated', color='Waste_Generated',
                   color_continuous_scale='YlOrBr',
                   title='<b>91. Agricultural Waste Generation</b>')

fig91.update_layout(height=500)
fig91.show()


91. Agricultural Waste...


### *92. Ecosystem Services Value*

In [30]:
print("\n92. Ecosystem Services...")

# Calculate ecosystem service values
services_data = (
    df.groupby(['Region', 'Irrigation_Type'], observed=False)
      .agg(
          Water_Productivity=('Water_Productivity', 'mean'),
          Mulching_Rate=('Mulching_Used', lambda s: s.eq('Yes').mean() * 100),
          Crop_Type=('Crop_Type', 'nunique')
      )
      .reset_index()
)


services_data['Eco_Value'] = (
    services_data['Water_Productivity'] * 40 +
    services_data['Mulching_Rate'] * 30 +
    services_data['Crop_Type'] * 30
)

fig92 = px.bar(services_data, x='Region', y='Eco_Value', color='Irrigation_Type',
               barmode='stack', title='<b>92. Ecosystem Service Value</b>',
               labels={'Eco_Value': 'Ecosystem Service Score'})

fig92.update_layout(height=500, plot_bgcolor='white')
fig92.show()


92. Ecosystem Services...


### *93. Climate Mitigation Potential*

In [32]:
print("\n93. Carbon Sequestration...")

# Carbon sequestration potential by practice
sequestration = {
    'mulching': 0.5,  # tonnes CO2 per ha per year
    'drip_irrigation': 0.3,
    'crop_diversity': 0.2
}

mitigation_data = (
    df.groupby('Region', observed=False)
      .agg(
          Mulching_Rate=('Mulching_Used', lambda s: s.eq('Yes').mean()),
          Modern_Rate=('Irrigation_Type', lambda s: s.isin(['Drip', 'Sprinkler']).mean()),
          Crop_Diversity=('Crop_Type', 'nunique')
      )
      .reset_index()
)

mitigation_data['Sequestration_Potential'] = (
    mitigation_data['Mulching_Rate'] * sequestration['mulching'] +
    mitigation_data['Modern_Rate'] * sequestration['drip_irrigation'] +
    (mitigation_data['Crop_Diversity'] / mitigation_data['Crop_Diversity'].max()) * sequestration['crop_diversity']
) * 100

fig93 = px.bar(mitigation_data.sort_values('Sequestration_Potential'), 
               x='Sequestration_Potential', y='Region', orientation='h',
               color='Sequestration_Potential', color_continuous_scale='Greens',
               title='<b>93. Carbon Sequestration Potential</b>',
               labels={'Sequestration_Potential': 'Sequestration Score'})

fig93.update_layout(height=500, plot_bgcolor='white')
fig93.show()


93. Carbon Sequestration...


### *94. Pollinator Dependency*

In [ ]:
print("\n94. Pollinator Analysis...")

# Pollinator dependency scores
pollinator_dep = {'Wheat': 0.1, 'Maize': 0.3, 'Cotton': 0.8, 
                  'Rice': 0.2, 'Sugarcane': 0.4, 'Potato': 0.6}

pollinator_data = df.groupby(['Region', 'Crop_Type'], observed=False).agg({
    'Field_Area_hectare': 'sum'
}).reset_index()

pollinator_data['Pollinator_Dependency'] = pollinator_data['Crop_Type'].map(pollinator_dep)
pollinator_data['Pollinator_Impact'] = pollinator_data['Field_Area_hectare'] * pollinator_data['Pollinator_Dependency']

fig94 = px.sunburst(pollinator_data, path=['Region', 'Crop_Type'], 
                    values='Pollinator_Impact', color='Pollinator_Dependency',
                    color_continuous_scale='RdYlBu',
                    title='<b>94. Pollinator Dependency by Crop & Region</b>')

fig94.update_layout(height=500)
fig94.show()


94. Pollinator Analysis...


### *95. Land Use Sustainability*

In [39]:
print("\n95. Land Use Sustainability...")

# Sustainability indicators
sustainability = df.groupby('Region', observed=False).agg({
    'Water_Productivity': 'mean',
    'Carbon_Footprint': 'mean',
    'Erosion_Risk': 'mean',
    'Pollution_Risk': 'mean'
}).reset_index()

# Calculate overall sustainability score (inverse of negative impacts)
sustainability['Sustainability_Score'] = (
    (sustainability['Water_Productivity'] / sustainability['Water_Productivity'].max() * 100) * 0.4 -
    (sustainability['Carbon_Footprint'] / sustainability['Carbon_Footprint'].max() * 100) * 0.2 -
    (sustainability['Erosion_Risk'] / sustainability['Erosion_Risk'].max() * 100) * 0.2 -
    (sustainability['Pollution_Risk'] / sustainability['Pollution_Risk'].max() * 100) * 0.2
)

sustainability['Sustainability_Score'] = sustainability['Sustainability_Score'].fillna(0)
sustainability['Sustainability_Score'] = pd.to_numeric(
    sustainability['Sustainability_Score'], errors='coerce'
).fillna(0)

size_col = 'Sustainability_Score_Size'
min_score = sustainability['Sustainability_Score'].min()
sustainability[size_col] = sustainability['Sustainability_Score'] - min_score  # >= 0

fig95 = px.scatter(
    sustainability,
    x='Water_Productivity',
    y='Carbon_Footprint',
    color='Sustainability_Score',
    size=size_col,
    hover_name='Region',
    color_continuous_scale='RdYlGn',
    title='<b>95. Land Use Sustainability Assessment</b><br>Green = More Sustainable',
    labels={'Water_Productivity': 'Water Productivity',
            'Carbon_Footprint': 'Carbon Footprint (kg CO2/ha)'}
)





fig95.update_layout(height=500, plot_bgcolor='white')
fig95.show()


95. Land Use Sustainability...
